In [ ]:
import fasttext
from huggingface_hub import hf_hub_download
from lingua import Language, LanguageDetectorBuilder
from langdetect import detect_langs
import numpy as np

In [ ]:
detector = LanguageDetectorBuilder.from_all_languages_with_cyrillic_script().build()
ft_model_path = hf_hub_download(repo_id="facebook/fasttext-language-identification", filename="model.bin")
ft_model = fasttext.load_model(ft_model_path)

In [ ]:
def langdetect_ru_conf(line):
    guesses = {guess.lang: guess.prob for guess in detect_langs(line)}
    if not 'ru' in guesses: guesses['ru'] = 0
    return np.round(guesses['ru'], 4)
def lingua_ru_conf(line):
    dtctr_ru_conf = detector.compute_language_confidence(line, Language.RUSSIAN)
    if dtctr_ru_conf < 0.01: dtctr_ru_conf = 0.01
    return np.round(dtctr_ru_conf, 4)
def fasttext_ru_conf(line):
    predict_langs, predict_confidence = ft_model.predict(line, k=3)
    if '__label__rus_Cyrl' in predict_langs: return np.round(predict_confidence[predict_langs.index('__label__rus_Cyrl')], 4)
    return 0

In [ ]:
def isRusLangText(text):
    langdetect_c, lingua_c, fasttext_c = langdetect_ru_conf(text), lingua_ru_conf(text), fasttext_ru_conf(text)
    return (langdetect_c >= 0.87 and lingua_c >= 0.87) or fasttext_c > 0.7

In [ ]:
# isRusLangText(input()) # тест